In [1]:
!pip install awscli-local


In [2]:
!pip install boto3

In [80]:
import boto3

# Configure boto3 to use the localstack endpoint for SQS and specify a mock region
sqs = boto3.client(
    'sqs',
    region_name='us-east-1', 
    endpoint_url='http://localhost:4566',
    aws_access_key_id='dummy_access_key', 
    aws_secret_access_key='dummy_secret_key'
)


In [81]:
def fetch_messages(queue_url, max_messages=10):
    """
    Fetch a batch of messages from the specified SQS queue.
    
    :param queue_url: The URL of the SQS queue.
    :param max_messages: The maximum number of messages to fetch in one request.
    :return: A list of messages.
    """
    response = sqs.receive_message(
        QueueUrl=queue_url,
        MaxNumberOfMessages=max_messages
    )
    return response.get('Messages', [])


In [82]:
QUEUE_URL = 'http://localhost:4566/000000000000/login-queue'
messages = fetch_messages(QUEUE_URL)
print(messages)


[{'MessageId': '182a0c29-0935-4b10-a9f0-665d1bf370b9', 'ReceiptHandle': 'ZWUwZmM4ZDUtZTUyMS00ZDU3LTlmYjktYzIwYjA5OWNkZDdhIGFybjphd3M6c3FzOnVzLWVhc3QtMTowMDAwMDAwMDAwMDA6bG9naW4tcXVldWUgMTgyYTBjMjktMDkzNS00YjEwLWE5ZjAtNjY1ZDFiZjM3MGI5IDE2OTYyMTkyNjUuNzM0MTc2Mg==', 'MD5OfBody': 'e4f1de8c099c0acd7cb05ba9e790ac02', 'Body': '{"user_id": "424cdd21-063a-43a7-b91b-7ca1a833afae", "app_version": "2.3.0", "device_type": "android", "ip": "199.172.111.135", "locale": "RU", "device_id": "593-47-5928"}'}, {'MessageId': '0f6b0334-7e00-410c-b36d-06f9bba80f66', 'ReceiptHandle': 'MjllMjhlMGItMjAzNi00N2E2LWI1ZDUtNGMxYzllOWQ3OTkwIGFybjphd3M6c3FzOnVzLWVhc3QtMTowMDAwMDAwMDAwMDA6bG9naW4tcXVldWUgMGY2YjAzMzQtN2UwMC00MTBjLWIzNmQtMDZmOWJiYTgwZjY2IDE2OTYyMTkyNjUuNzM0NzMxNw==', 'MD5OfBody': '347f6ce29bd4f361b13bc54c05d0a5fc', 'Body': '{"user_id": "c0173198-76a8-4e67-bfc2-74eaa3bbff57", "app_version": "0.2.6", "device_type": "ios", "ip": "241.6.88.151", "locale": "PH", "device_id": "104-25-0070"}'}, {'MessageId': 'a

In [83]:
bodies = [json.loads(message['Body']) for message in messages]


In [84]:
bodies

[{'user_id': '424cdd21-063a-43a7-b91b-7ca1a833afae',
  'app_version': '2.3.0',
  'device_type': 'android',
  'ip': '199.172.111.135',
  'locale': 'RU',
  'device_id': '593-47-5928'},
 {'user_id': 'c0173198-76a8-4e67-bfc2-74eaa3bbff57',
  'app_version': '0.2.6',
  'device_type': 'ios',
  'ip': '241.6.88.151',
  'locale': 'PH',
  'device_id': '104-25-0070'},
 {'user_id': '66e0635b-ce36-4ec7-aa9e-8a8fca9b83d4',
  'app_version': '2.2.1',
  'device_type': 'ios',
  'ip': '130.111.167.54',
  'locale': None,
  'device_id': '127-42-0862'},
 {'user_id': '181452ad-20c3-4e93-86ad-1934c9248903',
  'app_version': '0.96',
  'device_type': 'android',
  'ip': '118.79.6.245',
  'locale': 'ID',
  'device_id': '190-44-3099'},
 {'user_id': '60b9441c-e39d-406f-bba0-c7ff0e0ee07f',
  'app_version': '0.4.6',
  'device_type': 'android',
  'ip': '223.31.97.46',
  'locale': 'FR',
  'device_id': '149-99-5185'},
 {'user_id': '5082b1ae-6523-4e3b-a1d8-9750b4407ee8',
  'app_version': '3.7',
  'device_type': 'android',

In [85]:
def flatten_json(y):
    out = {}

    def flatten(x, name=''):
        if type(x) is dict:
            for a in x:
                flatten(x[a], name + a + '_')
        elif type(x) is list:
            i = 0
            for a in x:
                flatten(a, name + str(i) + '_')
                i += 1
        else:
            out[name[:-1]] = x

    flatten(y)
    return out


In [86]:
flattened_bodies = [flatten_json(body) for body in bodies]

In [87]:
import hashlib


In [88]:
def mask_data(input_data):
    """Return a SHA256 hashed version of the input data."""
    return hashlib.sha256(input_data.encode()).hexdigest()


In [89]:
for body in flattened_bodies:
    body['device_id'] = mask_data(body['device_id'])
    body['ip'] = mask_data(body['ip'])


In [90]:
flattened_bodies

#At this point, flattened_bodies will contain the transformed data with masked device_id and ip fields. 

[{'user_id': '424cdd21-063a-43a7-b91b-7ca1a833afae',
  'app_version': '2.3.0',
  'device_type': 'android',
  'ip': 'a6d0e2f27f6111e10b06790db42f34123e724aa0fd24b280f4a0ef5ee986784c',
  'locale': 'RU',
  'device_id': '4f00c1a807b673887c7af517d0df68e6b41aecf8cbec26c71fe4c580664669ed'},
 {'user_id': 'c0173198-76a8-4e67-bfc2-74eaa3bbff57',
  'app_version': '0.2.6',
  'device_type': 'ios',
  'ip': '7b03f7d723535706b4777384fc906d18a4376bb84cebb50dc22c6eb9bddf00cb',
  'locale': 'PH',
  'device_id': 'a857e702f98990716938a0d74c3dc2dc565e4448833e2cf91c6ab26fc0e9971f'},
 {'user_id': '66e0635b-ce36-4ec7-aa9e-8a8fca9b83d4',
  'app_version': '2.2.1',
  'device_type': 'ios',
  'ip': 'fa7fca28c658d75a751b60e262602e1b11f4149274af6ec0d8c82a8619a51437',
  'locale': None,
  'device_id': 'e84fb3e15175d0a2492de6c02a99595c1343db7321ad6bb5f62052edd00a84f8'},
 {'user_id': '181452ad-20c3-4e93-86ad-1934c9248903',
  'app_version': '0.96',
  'device_type': 'android',
  'ip': 'b21d1c922d9e9d1b913ade3265baa7fc43c757

In [91]:
!pip install psycopg2


In [92]:
import psycopg2


In [96]:
connection = psycopg2.connect(
    dbname="postgres",
    user="postgres",
    password="Tharun@2403",
    host="localhost",
    port="5432"
)
cursor = connection.cursor()


In [97]:
def insert_into_db(data, cursor):
    """
    Inserts data into the user_logins table.

    :param data: Dictionary containing user data.
    :param cursor: PostgreSQL cursor object.
    """
    insert_query = """
    INSERT INTO user_logins (user_id, device_type, masked_ip, masked_device_id, locale, app_version, create_date)
    VALUES (%s, %s, %s, %s, %s, %s, NOW())
    """
    cursor.execute(insert_query, (
        data["user_id"], 
        data["device_type"], 
        data["ip"],  # Masked IP
        data["device_id"],  # Masked device_id
        data.get("locale", "UNKNOWN"),  # Using "UNKNOWN" if locale is None or not present
        data["app_version"]
    ))


In [98]:
for body in flattened_bodies:
    insert_into_db(body, cursor)


In [99]:
connection.commit()
cursor.close()
connection.close()
